# Project Fatigue
- GitHub [link](https://github.com/romainmartinez/fatigue)

## todos
- keras/tensorflow: [link](https://medium.com/@williamkoehrsen/deep-neural-network-classifier-32c12ff46b6c)
- model optimization
- apply: [link](https://towardsdatascience.com/fine-tuning-a-classifier-in-scikit-learn-66e048c21e65)

In [1]:
# Common imports
import scipy.io as sio
import numpy as np
import os

# Figures
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

# to make this notebook's output stable across runs
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## 0. load data

In [2]:
DATA_PATH = os.path.join('.', 'data/')
DATA_FILE = 'DatabaseRPT.mat'
mat = sio.loadmat(os.path.join(DATA_PATH, DATA_FILE))['DataBaseRPT'][0, 0]

- label
    - `Y` (1, 162): 1 = prefatigue, 2 = fatigue

- used features
    - `AllX` (24, 162): 24 (6 DoF x 4 variables) x 162 (81 participants x 2 time points).
    - `CAssignAll` (1, 24): AllX column assignment

    - `Sex` (1, 162)
    - `Endurance` (1, 162)

- not used
    - `BestX` (7, 162): 7 (variables with SRM>0.8) x 162 (81 participants x 2 time points) matrix. Contains data only for the most responsive variables (SRM>0.8).
    - `CAssignBest` (1, 7): BestX column assignment.
    - `Age` (1, 162): too much NaN.
    - `Height` (1, 162): too much NaN.
    - `Weight` (1, 162) Too much NaN.
    - `SubjectKey` (1, 162): useless.
    - `SID` (1, 162): useless.

### create feature matrix

In [3]:
col_names = [i[0] for i in mat['CAssignAll'].flatten()]
# find variable with SRM > .8
srm_names = np.array([i[0] for i in mat['CAssignBest'].flatten()])
srm_idx = np.in1d(col_names, srm_names)

# add `AllX`
X = mat['AllX'].T

# add `Sex`
X = np.c_[X, mat['Sex'].T]
col_names.append('Sex')

# add `Endurance`
X = np.c_[X, mat['Endurance'].T]
col_names.append('Endurance')

col_names = np.array(col_names)

### create label vector

In [4]:
y = mat['Y'].T

### create report

In [5]:
import os.path
import pandas as pd

# create dataframe
df = pd.DataFrame(X, columns=col_names)
df['fatigue (label)'] = y

# generate pandas report
REPORT_FILENAME = './pandas_report.html'
if not os.path.isfile(REPORT_FILENAME):
    import pandas_profiling
    report = pandas_profiling.ProfileReport(df)
    report.to_file('./pandas_report.html')

### split data & shuffle

In [6]:
# split data & shuffle
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

## 1. Pipeline

In [7]:
df.head()

,AveElbFlexMean,ROMElbFlexMean,AveElbFlexSD,ROMElbFlexSD,AveShPlaneMean,ROMShPlaneMean,AveShPlaneSD,ROMShPlaneSD,AveShEleMean,ROMShEleMean,...,ROMTrYMean,AveTrYSD,ROMTrYSD,AveTrZMean,ROMTrZMean,AveTrZSD,ROMTrZSD,Sex,Endurance,fatigue (label)
0,105.983144,63.339618,2.219401,1.794134,23.518597,13.624799,1.338393,2.357813,80.274017,2.942674,...,13.629558,1.112671,1.977798,-0.550047,2.671822,0.228494,0.711787,2.0,2.0,1
1,106.166216,61.875226,2.239310,2.548458,22.851141,11.726534,1.182490,1.401127,77.897777,4.062635,...,15.078508,0.910697,1.003788,-0.208865,1.242553,0.541000,0.496108,2.0,2.0,2
2,100.914284,60.930029,1.482458,1.167593,27.289316,12.291697,1.223189,2.452705,83.932823,2.886935,...,14.311074,0.979720,2.070705,0.096174,0.934767,0.488540,0.317386,2.0,8.0,1
3,105.066549,59.524098,2.364182,1.849991,23.083816,7.519415,1.510000,3.503819,78.701025,2.997990,...,16.178023,1.418294,3.521951,-0.544067,1.885480,0.610577,0.851287,2.0,8.0,2
4,93.259333,81.531138,2.610755,3.632922,42.387152,27.789377,1.605530,2.027628,71.280986,8.000008,...,8.815273,0.709582,1.250228,-0.123442,1.065910,0.415764,0.404298,2.0,4.0,1


In [8]:
# custom class
from sklearn.base import BaseEstimator, TransformerMixin

# decomposition
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA, NMF

# scale
from sklearn.preprocessing import StandardScaler

# models
from xgboost import XGBClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# other
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
    
def get_categorical_cols(X):
    return X[:, np.logical_or(col_names == 'Sex', col_names == 'Endurance')]

def get_numerical_cols(X):
    return X[:, np.logical_and(col_names != 'Sex', col_names != 'Endurance')]

def get_high_srm_cols(X, srm_idx=srm_idx):
    return X[:, srm_idx]

In [9]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer

pipeline_categorical = Pipeline([
    ('selector', FunctionTransformer(get_categorical_cols, validate=False)),
    ('encoder', OneHotEncoder(sparse=False))
])

pipeline_numerical = Pipeline([
    ('selector', FunctionTransformer(get_numerical_cols, validate=False)),
    ('selecthighsrm', FunctionTransformer(get_high_srm_cols, validate=False)),
    ('scale', StandardScaler()),
    ('polyfeatures', PolynomialFeatures(degree=2))
])

pipeline_preprocessing = FeatureUnion([
    ('categorical', pipeline_categorical),
    ('numerical', pipeline_numerical)
])

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pprint
import warnings

models = {
    'XGBClassifier': XGBClassifier(),
    'ExtraTreeClassifier': ExtraTreeClassifier(),
    'LogisticRegression': LogisticRegression(),
    'GaussianNB': GaussianNB(),
    'SVC': SVC(), 
    'KNeighborsClassifier': KNeighborsClassifier()
}

param_grid = {
    'preprocessing__numerical__selecthighsrm': [None, FunctionTransformer(get_high_srm_cols, validate=False)],
    'preprocessing__numerical__polyfeatures': [None, PolynomialFeatures(degree=1), PolynomialFeatures(degree=2), PolynomialFeatures(degree=3)],
    'preprocessing__numerical__scale': [None, StandardScaler()],
    'pca': [None, PCA(n_components=0.99, svd_solver='full')]
}

scoring = ['accuracy', 'precision', 'recall', 'f1']

for model_name, model in models.items():
    print(f'{model_name}'.upper())
    
    # ignore xgboost warnings
    if model_name is 'XGBClassifier':
         warnings.simplefilter('ignore')
    else:
        warnings.simplefilter('default')
    
    pipeline_full = Pipeline([
        ('preprocessing', pipeline_preprocessing),
        ('pca', PCA(n_components=0.99, svd_solver='full')),
        ('classifier', model)
    ])
    
    grid_search = GridSearchCV(pipeline_full, param_grid, cv=5, scoring='accuracy',
                                verbose=1)
    grid_search.fit(X_train, y_train.ravel())
    
    print(f'\tbest score: {grid_search.best_score_:.3f}')
    print('\tbest params')
    pprint.pprint(grid_search.best_params_)
    
    y_pred = grid_search.best_estimator_.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)
    print(f'accuracy = {test_acc * 100}')
    
    print('-' * 10)

XGBCLASSIFIER
Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:   19.5s finished


	best score: 0.822
	best params
{'pca': PCA(copy=True, iterated_power='auto', n_components=0.99, random_state=None,
  svd_solver='full', tol=0.0, whiten=False),
 'preprocessing__numerical__polyfeatures': None,
 'preprocessing__numerical__scale': None,
 'preprocessing__numerical__selecthighsrm': FunctionTransformer(accept_sparse=False,
          func=<function get_high_srm_cols at 0x7f0afb3ab510>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)}
accuracy = 72.72727272727273
----------
EXTRATREECLASSIFIER
Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:    5.2s finished


	best score: 0.752
	best params
{'pca': None,
 'preprocessing__numerical__polyfeatures': PolynomialFeatures(degree=2, include_bias=True, interaction_only=False),
 'preprocessing__numerical__scale': None,
 'preprocessing__numerical__selecthighsrm': FunctionTransformer(accept_sparse=False,
          func=<function get_high_srm_cols at 0x7f0afb3ab510>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)}
accuracy = 60.60606060606061
----------
LOGISTICREGRESSION
Fitting 5 folds for each of 32 candidates, totalling 160 fits


...

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:    5.2s finished


	best score: 0.845
	best params
{'pca': PCA(copy=True, iterated_power='auto', n_components=0.99, random_state=None,
  svd_solver='full', tol=0.0, whiten=False),
 'preprocessing__numerical__polyfeatures': PolynomialFeatures(degree=2, include_bias=True, interaction_only=False),
 'preprocessing__numerical__scale': None,
 'preprocessing__numerical__selecthighsrm': FunctionTransformer(accept_sparse=False,
          func=<function get_high_srm_cols at 0x7f0afb3ab510>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)}
accuracy = 75.75757575757575
----------
SVC
Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:    5.5s finished


	best score: 0.791
	best params
{'pca': None,
 'preprocessing__numerical__polyfeatures': None,
 'preprocessing__numerical__scale': StandardScaler(copy=True, with_mean=True, with_std=True),
 'preprocessing__numerical__selecthighsrm': FunctionTransformer(accept_sparse=False,
          func=<function get_high_srm_cols at 0x7f0afb3ab510>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)}
accuracy = 75.75757575757575
----------
KNEIGHBORSCLASSIFIER
Fitting 5 folds for each of 32 candidates, totalling 160 fits
	best score: 0.806
	best params
{'pca': None,
 'preprocessing__numerical__polyfeatures': PolynomialFeatures(degree=2, include_bias=True, interaction_only=False),
 'preprocessing__numerical__scale': None,
 'preprocessing__numerical__selecthighsrm': FunctionTransformer(accept_sparse=False,
          func=<function get_high_srm_cols at 0x7f0afb3ab510>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:    5.6s finished


---
KERAS

In [6]:
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [13]:
X_train.shape

(129, 26)

In [19]:
from keras.models import Sequential
from keras.layers import Dense

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=26, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(26, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

NN = KerasClassifier(build_fn=create_model,
                     epochs=10,
                     batch_size=10,
                     verbose=1)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results = cross_val_score(NN, X, y, cv=kfold)

In [ ]:
NN.fit(X_train, y_train)